##### Exemplo de dados

In [ ]:
# COD_UF;COD_MUN;COD_ESPECIE;LATITUDE;LONGITUDE;NV_GEO_COORD
# 11;1100015;1;-11.929621;-61.999058;1
# 11;1100015;7;-11.929365;-61.999312;1
# 11;1100015;6;-11.929400;-61.999499;1
# 11;1100015;6;-11.929364;-61.999695;1
# 11;1100015;1;-11.929381;-61.999262;1
# 11;1100015;1;-11.930439;-61.999046;1
# 11;1100015;7;-11.930571;-61.999006;1
# 11;1100015;1;-11.930311;-61.998982;1
# 11;1100015;1;-11.930717;-61.999009;1
# 11;1100015;1;-11.930282;-61.999566;1

#### AJUSTE DE DADOS

In [1]:
import csv
import os

In [4]:
arquivos_csv = [arq for arq in os.listdir('arquivos/carga_python/') if arq.endswith('.csv')]

In [5]:
dados = []

for i in arquivos_csv:
    dir_arquivo = os.path.join('arquivos/carga_python', i)
    with open(dir_arquivo, 'r') as arquivo:
        arquivo_csv = csv.reader(arquivo, delimiter=';')
        next(arquivo_csv)
        data = [linha for linha in arquivo_csv]
        dados.extend(data)

In [7]:
print(f'{dados[0][0].strip()} {dados[0][1].strip()} {dados[0][2].strip()} {dados[0][3].strip()} {dados[0][4].strip()} {dados[0][5].strip()}')

41 4106803 1 -26.019385 -51.348250 1


In [8]:
dados_insert = []

for i in dados:
    dados_insert.append({'cod_uf': i[0], 'cod_mun' : i[1], 'cod_especie' : i[2], 'latitude' : i[3], 'longitude' : i[4], 'nv_geo_coord' : i[5]})

In [36]:
del dados

#### PARTE DB POSTGRESQL

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

url = URL.create(
    drivername="postgresql",
    username="postgres",
    host="localhost",
    database="postgres",
    password="123"
)

engine = create_engine(url)

In [2]:
connection = engine.connect()

In [3]:
from sqlalchemy import Column, Integer, Float
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

In [4]:
Base = declarative_base()

In [5]:
class Localidade(Base):
    __tablename__ = 'ibge'

    id = Column(Integer(), primary_key=True, autoincrement=True)
    cod_uf = Column(Integer(), nullable=True)
    cod_mun = Column(Integer(), nullable=True)
    cod_especie = Column(Integer(), nullable=True)
    latitude = Column(Float(), nullable=True)
    longitude = Column(Float(), nullable=True)
    nv_geo_coord = Column(Integer(), nullable=True)


In [6]:
Localidade.__table__

Table('ibge', MetaData(), Column('id', Integer(), table=<ibge>, primary_key=True, nullable=False), Column('cod_uf', Integer(), table=<ibge>), Column('cod_mun', Integer(), table=<ibge>), Column('cod_especie', Integer(), table=<ibge>), Column('latitude', Float(), table=<ibge>), Column('longitude', Float(), table=<ibge>), Column('nv_geo_coord', Integer(), table=<ibge>), schema=None)

In [27]:
Base.metadata.create_all(engine)

#### INSERT DE DADOS NO POSTGRESQL

In [ ]:
import gc

gc.collect()

In [7]:
Session = sessionmaker(bind=engine)
session = Session()

In [20]:
tamanho_lote = 10000

lotes = [dados_insert[i:i + tamanho_lote] for i in range(0, len(dados_insert), tamanho_lote)]

In [9]:
import time
from sqlalchemy import func

In [22]:
tic = time.perf_counter()

for l in lotes:
    session.bulk_insert_mappings(Localidade, l)
    session.commit()

toc = time.perf_counter()
tempo = (toc - tic)/60

In [10]:
print('----------------------------------------------------------------')
quantidade = session.query(func.count()).select_from(Localidade).scalar()
print(f'-- Total de dados registrados no DB: {quantidade}')
print('----------------------------------------------------------------')

----------------------------------------------------------------
-- Total de dados registrados no DB: 111102875
----------------------------------------------------------------


In [11]:
session.close()